# Model generation using doc2vec

## Introduction

Answering a research question based on information within a corpus of text relies on the ability to extract relevant subsets of the corpus (documents, parts of documents, etc.). The defintion of what constitutes a relevant subset may, however, not be immediately implementable in classical information retrieval terms such as keywords (which may also be perceived as limiting), and is not an intuitive starting point for researchers accustomed to working close to text, i.e. close-readng documents to identify and retrieve the information which interests them.

As an alternative to classical keyword defintion, and inspired by the approach of identifying relevant documents or passages using a more holistic assesment of their content, in the following we present the construction of a shallow neural network based model, trained on a user supplied corpus (which has been preprocessed by the user), aiming to encapsualte the content of a corpus element, with the goal, of enabling the user to retrieve elements of similar content by querying the corpus using the content of a seed element as numerically encoded by the model.

In the fllowing we construct the model using the `gensim` Python package and the doc2vec model framework it provides.

## Setup

Commented out the lines below for less verbosity

In [ ]:
# import logging # gensim progress
# logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

The following steps set up our environment

First import the standard and required framework packages

In [ ]:
import os, json
import numpy as np
import scipy as sp

Next import packages directly related to the model construction and serialization of model output

In [ ]:
import gensim
import pandas as pd
import import_ipynb
import corpus_reinferral
from gensim_progress import ProgressCallback

In [ ]:
corpus_file_path                      = os.path.join('/', 'data', 'model', 'tokenized_fragments.txt')
corpus_ids_file_path                  = os.path.join('/', 'data', 'model', 'fragment_ids.txt')
model_output_file_path                = os.path.join('/', 'data', 'model', 'doc2vec.d2v')
model_output_corpus_vectors_path      = os.path.join('/', 'data', 'model', 'corpus-vectors.pkl')
model_output_corpus_vectors_json_path = os.path.join('/', 'data', 'model', 'corpus-vectors.json')

Finally, as model construction is a process which requires a significant amout of computation, import packages to enable efficent usage of the available computational infrastructure, and specify allowed usage of infrastructure


In [ ]:
import multiprocessing

cores = multiprocessing.cpu_count()
usecores=np.int(3*cores/4)

## Utility Functions

Having setup the environment we next define a small number of utility functions which will enable us to import the (already preprocessed) corpus and subsequently train our model.

#### Function to read in corpus into input format supported by `gensim` `doc2vec`

This assumes that the corpus token file has been produced following the process encoded in the pre-processing notebook (), i.e. each corpus element (a 'document' in the sense of the doc2vec model) to be included in the model has been preprocessed and tokenized and is stored as a single string of tokens, with one element ('document') per line.

We note, that in many cases these elements/documents will, in fact, correspond to e.g. paragraphs of a larger document.

In [ ]:
def read_corpus(corpus_file):
    with open(corpus_file,'r') as tf:
        for i,text_line in enumerate(tf):
            tokens = text_line.split(' ')
            yield gensim.models.doc2vec.TaggedDocument(tokens,[i])
            

#### Function to read in corpus and list of corpus element file names, creating an object which supports inspecting and linking model output

In [ ]:
def read_corpus_lookup(corpus_ids_file, corpus_file) :
    with open(corpus_ids_file, 'r') as fnf, open(corpus_file,'r') as tf:
        i=0
        for (fn_line,tf_line) in zip(fnf,tf):
            yield ([i],[fn_line.rstrip()],[tf_line])
            i+=1

## Load corpus

With the utility functions defined we can now load the corpus

In [ ]:
corp = list(read_corpus(corpus_file_path))

and the lookup corpus

In [ ]:
corp_lookup = list(read_corpus_lookup(corpus_ids_file_path,corpus_file_path))

Unique identifiers for each corpus element can then be constructed from the lookup corpus.

In [ ]:
corp_ids =[]
for i in range(len(corp)):
    corp_ids.append(corp_lookup[i][1][0])

## Build Model

Having imported the corpus we can now build a model from it.

First we specify the model we want to bulid. In this case that is doc2vec with largely default settings, with modifications as specified in the following:

In [ ]:
vector_dimension=50
word_min_count=2
number_of_epochs=30

In [ ]:
callback = ProgressCallback(number_of_epochs)

In [ ]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=vector_dimension, min_count=word_min_count, epochs=number_of_epochs, workers=usecores, callbacks=[callback])

next, using the model, we build the vocabulary that will be used

In [ ]:
model.build_vocab(corp)

then, we train the model, timing the process 

In [ ]:
model.train(corp, total_examples=model.corpus_count, epochs=model.epochs)

and save the resulting trained model

In [ ]:
model.save(model_output_file_path)

## Reinfer corpus vectors 

Significant sections of the corpus vectors have been constructed during early epochs of the model training. Furthermore, in general, the corpora being modelled will be relatively small, so that individual instances of a derived/infered vector may be unstable and fluctuate in component dimensions. To adress this issue we reinfer the vector for each element of the corpus using the fully trained and frozen model multiple times, using the component wise median as the descriptive vector associated with a corpus element in further processing.

Use imported reinferral engine (corpus_reinferral.ipynb)

#### Reinferral settings

In [ ]:
number_reinferral_instances=100  #this corresponds to the default setting

### Execute reinferral 

In [ ]:
reinferred_corpus_vectors = corpus_reinferral.reinfer_corpus_single(corp,model,reinferral_instances=number_reinferral_instances)

Having reinferred the corpus vectors the results are saved in `numpy` binary format for potenial later (rapid) use.

### Create dataframe with mapping

The reinferred vectors save above require a separate mapping structure to ensure the correct association of a vector with the corresponding corpus element in a more general use scenario which might include reordering. To this end, we save a dataframe containing the reinferred vectors and the associated element names/ids. 

In [ ]:
data_list = list(map(list,zip(corp_ids,reinferred_corpus_vectors)))
vectorDF = pd.DataFrame(data_list,columns=['id','vector'])
vectorDF.to_pickle(model_output_corpus_vectors_path)

In [ ]:
data_dict={data_list_element[0]:data_list_element[1].tolist() for data_list_element in data_list}
with open(model_output_corpus_vectors_json_path,'w') as jf:
    jf.write(json.dumps(data_dict,indent=2))

## Summary

With the code above, we have built a doc2vec model on the ALREADY PREPROCCESSED corpus supplied, and have constructed averaged (i.e. numerically stablized) vectors in the model space for all corpus elements. Both the model and the vectors have been serialized for further use.